<a href="https://colab.research.google.com/github/julietamponti/tp2Datos-2C2021-/blob/main/1)_Consigna_33_terminado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***¿Dónde está la referencia geográfica más repetida en la tierra de toda la Wikipedia Español?***

Realizo la instalación de todo lo que utilizare en el desarrollo de la consulta.

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
#!apt install default-jre
#!apt install default-jdk
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 281.3 MB 41 kB/s 
     |████████████████████████████████| 198 kB 58.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=a5bfb92ee95d1f7b5a6b89e155feb93b46a18f34240f85136106124379a039e1
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hi

Importo librerias.

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd 

Creo el sparkSession.

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

Accedo a mi drive para llevar acabo la lectura del archivo.

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')
sqlContext = SQLContext(sc)

Mounted at /content/gdrive


/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


Realizo la lectura del archivo.

In [ ]:
geo = spark.read.format("csv").option("header", True).load('gdrive/My Drive/TP2 - Datos/geo_tags.csv')
rddGeo = geo.rdd

In [ ]:
rddGeo.take(2)

[Row(gt_id='158041', gt_page_id='4328020', gt_globe='earth', gt_primary='0', gt_lat='41.50749969', gt_lon='-5.81750011', gt_dim='1000', gt_type=None, gt_name=None, gt_country=None, gt_region=None),
 Row(gt_id='5844377', gt_page_id='4670424', gt_globe='earth', gt_primary='0', gt_lat='13.73799992', gt_lon='-89.29199982', gt_dim='10000', gt_type='city', gt_name=None, gt_country=None, gt_region=None)]

Limpio el RDD con el que voy a trabajar, y filtro por "globe" que me interesa, que en este caso es la tierra.

In [ ]:
rddGeo = rddGeo.filter(lambda x: x.gt_globe == 'earth' and x.gt_lon != None and x.gt_lat != None)

Realizo un map para quedarme con una estructura del estilo --> ((latitud,longitud),1), para luego realizar un reduceByKey y obtener el conjunto de longitud y latitud más repetido.

In [ ]:
rddGeo.map(lambda x: ((x[4],x[5]),1)).reduceByKey(lambda x,y:x+y).takeOrdered(1, lambda x:-x[1])

[(('-30.5', '-64.5'), 255)]

***Resultado***

Por lo tanto la referencia geografica mas repetida es ('-30.5', '-64.5')